####  Relevant libraries

In [1]:
import googlemaps
import getpass
import pandas as pd
import numpy as np

####  Google api key & prepare raw addresses

Google api key to query maps, geolocations and more. Get yours at the [google developers section](https://console.cloud.google.com/), make sure to enable the relevant queries (gelocating in this case) and keep it safe.

In [10]:
gmaps = googlemaps.Client(key=getpass.getpass())

········


In [2]:
data = pd.read_excel('../potential_data/election-25_kalpi-places_kalpies_list_10_7_nagish.xlsx')

In [3]:
data.head()

סמל ועדה  שם ועדה  סמל ישוב בחירות שם ישוב בחירות  סמל קלפי  סמל רכוז  \
0         1  ירושלים             3000        ירושלים         1        14   
1         1  ירושלים             3000        ירושלים         2       414   
2         1  ירושלים             3000        ירושלים         3        18   
3         1  ירושלים             3000        ירושלים         4        59   
4         1  ירושלים             3000        ירושלים         5       383   

             כתובת קלפי                       מקום קלפי נגישה נגישה מיוחדת  \
0  מעגלי הרי"ם לוין,27   ביה"ס בית יעקב הצפון (סנהדריה)                      
1       שמואל הנביא,85         ת"ת שערי תורה (למען אחי)     כ                
2           מגן האלף,1         סמינר בית יעקב - עטרת חן                      
3         פישל אהרן,29               ת"ת זיכרו תורת משה     כ                
4        שבטי ישראל,27                      משרד החינוך                      

   בוחרי כנסת בפועל  
0               743  
1               453  
2               614  
3               565  
4               543

In [4]:
rehovot_polls = (data[data['שם ישוב בחירות'] == "רחובות"]
 ['מקום קלפי']
 .unique()
.tolist()
)

In [5]:
rehovot_polls[1:5]

['מתנ"ס כפר גבירול ישן (בית תנוב',
 'חט"צ ע"ש בגין',
 'בי"ס עץ חיים',
 'תלמוד תורה ממד הראי"ה (הדרים)']

Great, we have a list of our polling locations, let's go ahead and collect their geolocations.

#### Function to geocode the address & iterate across the list

We'll write up a function to query an addresses since the 'gmaps.gecode' returns a json format object. in case nothing returns it's an emtpy list, so we want to handle that too. other than return the relevant geometric location. Thanks to [this StackOverflow response](https://stackoverflow.com/questions/37311687/extracting-lat-lon-from-geocode-result-list-with-python-google-maps-api) for the quick function.

In [21]:
def geocode_address(loc):
    geocode_result = gmaps.geocode(loc + ', רחובות')
    if not geocode_result:
        location = {
        'address': loc,
        'lat' : np.NaN,
        'lon': np.NaN
    }
    else:
        lat = geocode_result[0]["geometry"]["location"]["lat"]
        lon = geocode_result[0]["geometry"]["location"]["lng"]
        location = {
            'address': loc,
            'lat' : lat,
            'lon': lon
        }
    df = pd.DataFrame([location])
    return df

In [22]:
geocode_address(rehovot_polls[0]) # test output

address       lat        lon
0  בי"ס נבון  31.89273  34.776944

In [23]:
lst = []

In [24]:
for i in range(len(rehovot_polls)):
    lst.append(geocode_address(rehovot_polls[i]))

In [25]:
addresses = pd.concat(lst)
addresses.head()

address        lat        lon
0                       בי"ס נבון  31.892730  34.776944
0  מתנ"ס כפר גבירול ישן (בית תנוב  31.894527  34.777092
0                   חט"צ ע"ש בגין  31.902192  34.825207
0                    בי"ס עץ חיים  31.885227  34.786213
0   תלמוד תורה ממד הראי"ה (הדרים)  31.892773  34.811272

In [27]:
addresses.to_csv('polling_station_rehovot.csv')

Voila! Enjoy and lookout for the graph using it — probably day 1 😉